# Kryptologie Lab - Übung 03

In [1]:
import numpy as np

In [2]:
encryptGaloisMatrix = np.array([[2,3,1,1],[1,2,3,1],[1,1,2,3],[3,1,1,2]])
decryptGaloisMatrix = np.array([[14,11,13,9],[9,14,11,13],[13,9,14,11],[11,13,9,14]])

## Init Galoiskörper-Operationen

In [4]:
def gfAddition(a,b):
    return np.bitwise_xor(a,b)

def gfMuliply(a,b):
    # to fill
    pass

def xTimes(a):
    t = a << 1
    if (a & 1 << 7):
        t = t ^ 0x1b
    return t

## Init algorithm steps

In [22]:
def initBlock(textblock):
    chars = [ord(c) for c in textblock]
    return np.reshape(np.array(chars, dtype=np.dtype('B')), (4,4),order="F")

def blockToText(textblock):
    text = textblock.transpose().flatten()
    toChar = lambda c: chr(c)
    return "".join(np.array([toChar(c) for c in text]))

def addRoundKey(textblock, key):
    chars = [ord(c) for c in key]
    key = np.reshape(np.array(chars, dtype=np.dtype('B')), (4,4),order="F")
    return np.bitwise_xor(textblock, key)

def subBytes(textblock, reverse=False):
    # to fill
    return textblock

def shiftRows(textblock, reverse=False):
    for i in range(4):
        if reverse:
            shiftAmount = i
        else:
            shiftAmount = -i
        textblock[i] = np.roll(textblock[i], shiftAmount)
    return textblock

def mixColumns(textblock, reverse=False):
    if reverse:
        matrix = decryptGaloisMatrix
    else:
        matrix = encryptGaloisMatrix
    for column in range(4):
        textblock[column] = subColumn(textblock[column], matrix)
    return textblock

def subColumn(inputColumn, matrix):
    outputColumn = np.copy(inputColumn)
    for row in range(4):
        val = 0
        for column in range(4):
            val = gfAddition(val, gfMuliply(matrix[row, column], inputColumn[column]))
        outputColumn[row] = val
    return outputColumn

In [23]:
block = initBlock("ABCDEFGHIJKLMNOP")
blockToText(block)

'ABCDEFGHIJKLMNOP'

## Init algorithm process

In [5]:
def encryptBlock(textblock, keys):
    textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[0])
    for i in range(1, 10):
        textblock = subBytes(textblock)
        textblock = shiftRows(textblock)
        textblock = mixColumns(textblock)
        textblock = addRoundKey(textblock, keys[i])
    textblock = subBytes(textblock)
    textblock = shiftRows(textblock)
    textblock = addRoundKey(textblock, keys[10])
    return blockToText(textblock)

def decryptBlock(textblock, keys):
    textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[0])
    for i in range(1, 10):
        textblock = subBytes(textblock, reverse=True)
        textblock = shiftRows(textblock, reverse=True)
        textblock = mixColumns(textblock, reverse=True)
        textblock = addRoundKey(textblock, keys[i])
    textblock = subBytes(textblock, reverse=True)
    textblock = shiftRows(textblock, reverse=True)
    textblock = addRoundKey(textblock, keys[10])
    return blockToText(textblock)

## Init block splitting

In [11]:
def encrypt(text, keys):
    blocks = [[text[i:i+16]] for i in range(0, len(text), 16)]
    return "".join([encryptBlock(block, keys) for block in blocks])

def decrypt(text, keys):
    blocks = [[text[i:i+16]] for i in range(0, len(text), 16)]
    return "".join([decryptBlock(block, keys) for block in blocks])

[['Hallo ich heiße '], ['matti und ich bi'], ['n leider nicht l'], ['ustig. Geht das '], ['hier noch viel w'], ['eiter']]


## Testing